In [1]:
from skimage.io import imread, imsave, imshow
import numpy

im = imread('img.png')

In [2]:
def extend_height(image, extended_image, width, height, addition = 0):
    for h in range(height - 1, -1, -1):
        for i in range(image.shape[1] + addition):
            extended_image[h, i + width] \
                = extended_image[h + 1 : h + 1 + height, i : i + height * 2 + 1].mean(axis=0).mean(axis=0)
            extended_image[-h - 1, i + width] \
                = extended_image[-h - 1 - height : -h - 1, i : i + height * 2 + 1].mean(axis=0).mean(axis=0)

In [3]:
def extend_width(image, extended_image, width, height, addition = 0):
    for w in range(width - 1, -1, -1):
        for i in range(image.shape[0] + addition):
            extended_image[i + height, w] \
                = extended_image[i : i + height * 2 + 1, w + 1 : w + 1 + width].mean(axis = 0).mean(axis = 0)
            extended_image[i + height, -w - 1] \
                = extended_image[i : i + height * 2 + 1, -w - 1 - width : -w - 1].mean(axis = 0).mean(axis = 0)

In [4]:
def extend_image(image, width = 1, height = 1):
    shape = image.shape

    fatting = width * image.shape[0] < height * image.shape[1]

    if fatting:
        width = 2 * width
    else:
        height = 2 * height

    extended_image = numpy.ones((shape[0] + height * 2, shape[1] + width * 2, shape[2]), numpy.uint8)
    extended_image[:] = 256 / 2
    extended_image[height : -height, width : -width] = image

    if fatting:
        extend_width(image, extended_image, width, height)
        width = int(width / 2)
        extend_height(image, extended_image, width, height, width * 2)
        return extended_image[ : , width : -width]
    else:
        extend_height(image, extended_image, width, height)
        height = int(height / 2)
        extend_width(image, extended_image, width, height, height * 2)
        return extended_image[height : -height, : ]

In [5]:
imsave('ninja.png', extend_image(im, 200, 100))